In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# For Neural Networks
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import os

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("="*80)
print("STREAMFLOW DISCHARGE FORECASTING - FIXED IMPLEMENTATION")
print("="*80)
print("Based on: Xiao & You (Stanford University)")
print("Models: Standard Feed-Forward NN and LSTM")
print("Fixes Applied:")
print("  ✓ Removed early stopping - full 200 epochs training")
print("  ✓ Using SGD optimizer (not Adam) for Standard NN")
print("  ✓ Proper best model selection based on validation RMSE")
print("  ✓ Correct activation functions (sigmoid for hidden, linear for output)")
print("="*80)

STREAMFLOW DISCHARGE FORECASTING - FIXED IMPLEMENTATION
Based on: Xiao & You (Stanford University)
Models: Standard Feed-Forward NN and LSTM
Fixes Applied:
  ✓ Removed early stopping - full 200 epochs training
  ✓ Using SGD optimizer (not Adam) for Standard NN
  ✓ Proper best model selection based on validation RMSE
  ✓ Correct activation functions (sigmoid for hidden, linear for output)


In [2]:
print("\n" + "="*60)
print("STEP 1: Loading Dataset...")
print("="*60)

df = pd.read_csv("dataset_with_combined_lags.csv")

print("\nFirst few rows of dataset:")
print(df.head())

# Convert datetime
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')

if df['datetime'].isna().any():
    print(f"Warning: {df['datetime'].isna().sum()} datetime values couldn't be parsed")
    df = df.dropna(subset=['datetime'])

# Remove missing values
print(f"\nChecking for missing values...")
print(f"  Missing in PRECTOTCORR: {df['PRECTOTCORR'].isna().sum()}")
print(f"  Missing in discharge_cfs: {df['discharge_cfs'].isna().sum()}")

df = df.dropna(subset=['PRECTOTCORR', 'discharge_cfs'])

print(f"\n✓ Dataset loaded successfully")
print(f"  Shape: {df.shape}")
print(f"  Time period: {df['datetime'].min()} to {df['datetime'].max()}")
print(f"  Total days: {len(df)}")

print(f"\nData Statistics:")
print(f"  Precipitation (PRECTOTCORR):")
print(f"    Min: {df['PRECTOTCORR'].min():.2f}, Max: {df['PRECTOTCORR'].max():.2f}")
print(f"    Mean: {df['PRECTOTCORR'].mean():.2f}, Std: {df['PRECTOTCORR'].std():.2f}")
print(f"  Discharge (discharge_cfs):")
print(f"    Min: {df['discharge_cfs'].min():.2f}, Max: {df['discharge_cfs'].max():.2f}")
print(f"    Mean: {df['discharge_cfs'].mean():.2f}, Std: {df['discharge_cfs'].std():.2f}")



STEP 1: Loading Dataset...

First few rows of dataset:
     datetime  PRECTOTCORR_z  discharge_cfs_z  discharge_cfs_z_lag1  \
0  01-01-2003      -0.394675         1.265288              1.265288   
1  02-01-2003      -0.427561         1.485855              1.265288   
2  03-01-2003      -0.458099         1.394400              1.485855   
3  04-01-2003      -0.458099         0.791876              1.394400   
4  05-01-2003      -0.453401         0.173212              0.791876   

   discharge_cfs_z_lag2  discharge_cfs_z_lag3  discharge_cfs_z_lag4  \
0              1.265288              1.265288              1.265288   
1              1.265288              1.265288              1.265288   
2              1.265288              1.265288              1.265288   
3              1.485855              1.265288              1.265288   
4              1.394400              1.485855              1.265288   

   discharge_cfs_z_lag5  PRECTOTCORR_z_lag1  PRECTOTCORR_z_lag2  \
0              1.265288

KeyError: 'PRECTOTCORR'

In [ ]:
print("\n" + "="*60)
print("STEP 2: Data Normalization (by Maximum Value)...")
print("="*60)

precipitation = df['PRECTOTCORR'].values
discharge = df['discharge_cfs'].values

# Normalize by maximum value (as per paper)
precip_max = precipitation.max()
discharge_max = discharge.max()

precip_normalized = precipitation / precip_max
discharge_normalized = discharge / discharge_max

print(f"✓ Precipitation max: {precip_max:.4f}")
print(f"✓ Discharge max: {discharge_max:.2f} ft³/s")
print(f"✓ Data normalized to [0, 1] range")
print(f"\n📌 Paper uses: Rainfall max = 6.8 inch, Discharge max = 29700 ft³/s")
print(f"📌 Your data: Check if values are comparable")


STEP 2: Data Normalization (by Maximum Value)...
✓ Precipitation max: 86.1700
✓ Discharge max: 29700.00 ft³/s
✓ Data normalized to [0, 1] range

📌 Paper uses: Rainfall max = 6.8 inch, Discharge max = 29700 ft³/s
📌 Your data: Check if values are comparable


In [ ]:
print("\n" + "="*60)
print("STEP 3: Creating Lagged Features...")
print("="*60)

def create_lagged_features(precip, discharge, lag_precip, lag_discharge):
    """
    Create lagged features: Q(t) = f{R(t-1),...,R(t-m), Q(t-1),...,Q(t-n)}
    """
    n_samples = len(precip)
    max_lag = max(lag_precip, lag_discharge)

    n_features = lag_precip + lag_discharge
    X = np.zeros((n_samples - max_lag, n_features))
    y = np.zeros(n_samples - max_lag)

    for i in range(max_lag, n_samples):
        # Lagged precipitation: R(t-1), R(t-2), ..., R(t-m)
        precip_features = [precip[i-j] for j in range(1, lag_precip + 1)]

        # Lagged discharge: Q(t-1), Q(t-2), ..., Q(t-n)
        discharge_features = [discharge[i-j] for j in range(1, lag_discharge + 1)]

        # Combine features
        X[i - max_lag, :] = precip_features + discharge_features

        # Target: Q(t)
        y[i - max_lag] = discharge[i]

    valid_indices = np.arange(max_lag, n_samples)
    return X, y, valid_indices

# Define lag combinations (as per paper)
lag_combinations_standard = [
    (2,3), (3,4), (4,5), (5,6), (6,7),  # n-m=1
    (2,4), (3,5), (4,6), (5,7),         # n-m=2
    (2,5), (3,6), (4,7),                # n-m=3
]

lag_combinations_lstm = [(i,i) for i in range(2,8)]  # m=n for LSTM

print(f"✓ Standard NN lag combinations: {len(lag_combinations_standard)}")
print(f"✓ LSTM lag combinations: {len(lag_combinations_lstm)}")



STEP 3: Creating Lagged Features...
✓ Standard NN lag combinations: 12
✓ LSTM lag combinations: 6


In [ ]:
print("\n" + "="*60)
print("STEP 4: Temporal Train-Test Split...")
print("="*60)

train_ratio = 0.8

def split_data(X, y, valid_indices, train_ratio=0.8):
    """Split data temporally (chronological order preserved)"""
    split_idx = int(len(X) * train_ratio)

    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    train_indices = valid_indices[:split_idx]
    test_indices = valid_indices[split_idx:]

    return X_train, X_test, y_train, y_test, train_indices, test_indices

print(f"✓ Train ratio: {train_ratio*100:.0f}%")
print(f"✓ Using temporal split (paper: 2003-2007 train, 2008-2012 validation)")



STEP 4: Temporal Train-Test Split...
✓ Train ratio: 80%
✓ Using temporal split (paper: 2003-2007 train, 2008-2012 validation)


In [ ]:
print("\n" + "="*60)
print("STEP 5: Building Neural Network Models...")
print("="*60)

def build_standard_nn(input_size, hidden_size=12):
    """
    Standard Feed-Forward Neural Network (3-layer)
    - Hidden layer: SIGMOID activation (as per paper)
    - Output layer: LINEAR (no activation, as per paper)
    """
    model = Sequential([
        Dense(hidden_size, activation='sigmoid', input_shape=(input_size,)),
        Dense(1, activation='linear')
    ])
    return model

def build_lstm_model(sequence_length, hidden_size=12):
    """
    LSTM Model (3-layer)
    - LSTM layer with specified hidden size
    - Output: dense linear
    """
    model = Sequential([
        LSTM(hidden_size, input_shape=(sequence_length, 2), return_sequences=False),
        Dense(1, activation='linear')
    ])
    return model

print("✓ Standard NN: Input → Hidden(sigmoid) → Output(linear)")
print("✓ LSTM: Input → LSTM → Output(linear)")


STEP 5: Building Neural Network Models...
✓ Standard NN: Input → Hidden(sigmoid) → Output(linear)
✓ LSTM: Input → LSTM → Output(linear)


In [ ]:
print("\n" + "="*60)
print("STEP 6: Hyperparameter Search & Model Training...")
print("="*60)

results_standard = []
results_lstm = []

hidden_sizes = [4, 8, 12, 16, 20, 24, 28]

# Create temp directory for model checkpoints
os.makedirs('temp_models', exist_ok=True)

print("\n" + "-"*60)
print("Training Standard Feed-Forward Neural Networks...")
print("-"*60)
print("Key changes from previous version:")
print("  • Using SGD optimizer (lr=0.009) instead of Adam")
print("  • Training for FULL 200 epochs (no early stopping)")
print("  • Saving best model based on validation loss (.keras format)")
print("-"*60)

model_count = 0
total_models_standard = len(lag_combinations_standard) * len(hidden_sizes)

for lag_precip, lag_discharge in lag_combinations_standard:
    print(f"\nLag config: m={lag_precip}, n={lag_discharge}")

    # Create features
    X, y, valid_indices = create_lagged_features(
        precip_normalized, discharge_normalized,
        lag_precip, lag_discharge
    )

    # Split data
    X_train, X_test, y_train, y_test, train_idx, test_idx = split_data(
        X, y, valid_indices, train_ratio
    )

    input_size = X_train.shape[1]

    for hidden_size in hidden_sizes:
        model_count += 1
        print(f"  [{model_count}/{total_models_standard}] Hidden={hidden_size}", end=" ", flush=True)

        # Build model
        model = build_standard_nn(input_size, hidden_size)

        # FIXED: Using SGD with learning rate 0.009 (as per paper)
        model.compile(
            optimizer=SGD(learning_rate=0.009, momentum=0.0),
            loss='mse',
            metrics=['mae']
        )

        # Save best model checkpoint (using .keras format to avoid warnings)
        checkpoint_path = f'temp_models/standard_m{lag_precip}_n{lag_discharge}_h{hidden_size}.keras'
        checkpoint = ModelCheckpoint(
            checkpoint_path,
            monitor='val_loss',
            save_best_only=True,
            mode='min',
            verbose=0
        )

        # FIXED: Train for FULL 200 epochs without early stopping
        history = model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=200,
            batch_size=32,
            callbacks=[checkpoint],
            verbose=0
        )

        # Load best model
        try:
            best_model = keras.models.load_model(checkpoint_path)
        except:
            # Fallback if loading fails
            best_model = model

        # Get predictions
        y_pred_test = best_model.predict(X_test, verbose=0).flatten()

        # Denormalize
        y_test_actual = y_test * discharge_max
        y_pred_test_actual = y_pred_test * discharge_max

        # Calculate metrics
        test_rmse = np.sqrt(mean_squared_error(y_test_actual, y_pred_test_actual))
        relative_rmse = (test_rmse / discharge_max) * 100
        test_r2 = r2_score(y_test_actual, y_pred_test_actual)

        # Peak vs Non-peak analysis (threshold = 1500 as per paper)
        threshold = 1500
        peak_mask = y_test_actual > threshold
        non_peak_mask = ~peak_mask

        peak_rmse_rel = 0
        non_peak_rmse_rel = 0

        if peak_mask.sum() > 0:
            peak_rmse = np.sqrt(mean_squared_error(
                y_test_actual[peak_mask],
                y_pred_test_actual[peak_mask]
            ))
            peak_rmse_rel = (peak_rmse / discharge_max) * 100

        if non_peak_mask.sum() > 0:
            non_peak_rmse = np.sqrt(mean_squared_error(
                y_test_actual[non_peak_mask],
                y_pred_test_actual[non_peak_mask]
            ))
            non_peak_rmse_rel = (non_peak_rmse / discharge_max) * 100

        print(f"→ RMSE: {relative_rmse:.2f}%")

        # Store results
        results_standard.append({
            'Lag_Precip': lag_precip,
            'Lag_Discharge': lag_discharge,
            'Hidden_Size': hidden_size,
            'Test_RMSE': test_rmse,
            'Relative_RMSE': relative_rmse,
            'Test_R2': test_r2,
            'Peak_RMSE_Relative': peak_rmse_rel,
            'NonPeak_RMSE_Relative': non_peak_rmse_rel,
            'y_pred_test': y_pred_test_actual,
            'y_test': y_test_actual,
            'test_indices': test_idx,
            'dates': df['datetime'].iloc[test_idx].values,
            'best_epoch': np.argmin(history.history['val_loss']) + 1
        })

        # Clean up checkpoint file
        try:
            if os.path.exists(checkpoint_path):
                os.remove(checkpoint_path)
        except:
            pass

print("\n" + "-"*60)
print("Training LSTM Models...")
print("-"*60)
print("Key changes:")
print("  • Using Adam optimizer (lr=0.001) as per paper for LSTM")
print("  • Training for FULL 200 epochs")
print("  • Using .keras format to avoid warnings")
print("-"*60)

model_count = 0
total_models_lstm = len(lag_combinations_lstm) * len(hidden_sizes)

for lag, _ in lag_combinations_lstm:
    print(f"\nLag config: m=n={lag}")

    # Create LSTM data: (samples, timesteps=lag, features=2)
    max_lag = lag
    n_samples = len(precip_normalized) - max_lag

    X_lstm = np.zeros((n_samples, max_lag, 2))
    y_lstm = np.zeros(n_samples)

    for i in range(max_lag, len(precip_normalized)):
        for t in range(max_lag):
            X_lstm[i - max_lag, t, 0] = precip_normalized[i - (max_lag - t)]
            X_lstm[i - max_lag, t, 1] = discharge_normalized[i - (max_lag - t)]
        y_lstm[i - max_lag] = discharge_normalized[i]

    valid_indices = np.arange(max_lag, len(precip_normalized))

    # Split
    split_idx = int(len(X_lstm) * train_ratio)
    X_train_lstm = X_lstm[:split_idx]
    X_test_lstm = X_lstm[split_idx:]
    y_train_lstm = y_lstm[:split_idx]
    y_test_lstm = y_lstm[split_idx:]
    test_idx = valid_indices[split_idx:]

    for hidden_size in hidden_sizes:
        model_count += 1
        print(f"  [{model_count}/{total_models_lstm}] Hidden={hidden_size}", end=" ", flush=True)

        # Build LSTM model
        model = build_lstm_model(sequence_length=max_lag, hidden_size=hidden_size)

        # Compile with Adam (lr=0.001 as per paper for LSTM)
        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )

        # Save best model checkpoint (using .keras format)
        checkpoint_path = f'temp_models/lstm_lag{lag}_h{hidden_size}.keras'
        checkpoint = ModelCheckpoint(
            checkpoint_path,
            monitor='val_loss',
            save_best_only=True,
            mode='min',
            verbose=0
        )

        # Train for full 200 epochs
        history = model.fit(
            X_train_lstm, y_train_lstm,
            validation_data=(X_test_lstm, y_test_lstm),
            epochs=200,
            batch_size=32,
            callbacks=[checkpoint],
            verbose=0
        )

        # Load best model
        try:
            best_model = keras.models.load_model(checkpoint_path)
        except:
            # Fallback if loading fails
            best_model = model

        # Predictions
        y_pred_test = best_model.predict(X_test_lstm, verbose=0).flatten()

        # Denormalize
        y_test_actual = y_test_lstm * discharge_max
        y_pred_test_actual = y_pred_test * discharge_max

        # Calculate metrics
        test_rmse = np.sqrt(mean_squared_error(y_test_actual, y_pred_test_actual))
        relative_rmse = (test_rmse / discharge_max) * 100
        test_r2 = r2_score(y_test_actual, y_pred_test_actual)

        # Peak analysis
        threshold = 1500
        peak_mask = y_test_actual > threshold
        non_peak_mask = ~peak_mask

        peak_rmse_rel = 0
        non_peak_rmse_rel = 0

        if peak_mask.sum() > 0:
            peak_rmse = np.sqrt(mean_squared_error(
                y_test_actual[peak_mask],
                y_pred_test_actual[peak_mask]
            ))
            peak_rmse_rel = (peak_rmse / discharge_max) * 100

        if non_peak_mask.sum() > 0:
            non_peak_rmse = np.sqrt(mean_squared_error(
                y_test_actual[non_peak_mask],
                y_pred_test_actual[non_peak_mask]
            ))
            non_peak_rmse_rel = (non_peak_rmse / discharge_max) * 100

        print(f"→ RMSE: {relative_rmse:.2f}%")

        # Store results
        results_lstm.append({
            'Lag_Precip': lag,
            'Lag_Discharge': lag,
            'Hidden_Size': hidden_size,
            'Test_RMSE': test_rmse,
            'Relative_RMSE': relative_rmse,
            'Test_R2': test_r2,
            'Peak_RMSE_Relative': peak_rmse_rel,
            'NonPeak_RMSE_Relative': non_peak_rmse_rel,
            'y_pred_test': y_pred_test_actual,
            'y_test': y_test_actual,
            'test_indices': test_idx,
            'dates': df['datetime'].iloc[test_idx].values,
            'best_epoch': np.argmin(history.history['val_loss']) + 1
        })

        # Clean up
        try:
            if os.path.exists(checkpoint_path):
                os.remove(checkpoint_path)
        except:
            pass

# Clean up temp directory
try:
    if os.path.exists('temp_models') and not os.listdir('temp_models'):
        os.rmdir('temp_models')
except:
    pass

# Convert to DataFrames
results_standard_df = pd.DataFrame(results_standard)
results_lstm_df = pd.DataFrame(results_lstm)

# Find best models
best_standard = results_standard_df.loc[results_standard_df['Relative_RMSE'].idxmin()]
best_lstm = results_lstm_df.loc[results_lstm_df['Relative_RMSE'].idxmin()]

print("\n" + "="*60)
print("BEST MODELS FOUND")
print("="*60)
print(f"\n🏆 Best Standard NN:")
print(f"   Lag Precip (m): {best_standard['Lag_Precip']}")
print(f"   Lag Discharge (n): {best_standard['Lag_Discharge']}")
print(f"   Hidden Size: {best_standard['Hidden_Size']}")
print(f"   Overall RMSE: {best_standard['Relative_RMSE']:.2f}%")
print(f"   Peak RMSE: {best_standard['Peak_RMSE_Relative']:.2f}%")
print(f"   Non-Peak RMSE: {best_standard['NonPeak_RMSE_Relative']:.2f}%")
print(f"   R² Score: {best_standard['Test_R2']:.4f}")
print(f"   Best Epoch: {best_standard['best_epoch']}")

print(f"\n🏆 Best LSTM:")
print(f"   Lag (m=n): {best_lstm['Lag_Precip']}")
print(f"   Hidden Size: {best_lstm['Hidden_Size']}")
print(f"   Overall RMSE: {best_lstm['Relative_RMSE']:.2f}%")
print(f"   Peak RMSE: {best_lstm['Peak_RMSE_Relative']:.2f}%")
print(f"   Non-Peak RMSE: {best_lstm['NonPeak_RMSE_Relative']:.2f}%")
print(f"   R² Score: {best_lstm['Test_R2']:.4f}")
print(f"   Best Epoch: {best_lstm['best_epoch']}")

print(f"\n📊 Paper's Best Results for Comparison:")
print(f"   Standard NN: Overall=0.23%, Peak=0.63%, Non-Peak=0.07%")
print(f"   LSTM: Overall=2.43%, Peak=6.78%, Non-Peak=0.54%")


STEP 6: Hyperparameter Search & Model Training...

------------------------------------------------------------
Training Standard Feed-Forward Neural Networks...
------------------------------------------------------------
Key changes from previous version:
  • Using SGD optimizer (lr=0.009) instead of Adam
  • Training for FULL 200 epochs (no early stopping)
  • Saving best model based on validation loss (.keras format)
------------------------------------------------------------

Lag config: m=2, n=3
  [1/84] Hidden=4 → RMSE: 5.20%
  [2/84] Hidden=8 → RMSE: 6.06%
  [3/84] Hidden=12 → RMSE: 5.35%
  [4/84] Hidden=16 → RMSE: 5.00%
  [5/84] Hidden=20 → RMSE: 4.21%
  [6/84] Hidden=24 → RMSE: 5.48%
  [7/84] Hidden=28 → RMSE: 5.29%

Lag config: m=3, n=4
  [8/84] Hidden=4 → RMSE: 6.40%
  [9/84] Hidden=8 → RMSE: 5.55%
  [10/84] Hidden=12 → RMSE: 5.61%
  [11/84] Hidden=16 → RMSE: 5.15%
  [12/84] Hidden=20 → RMSE: 4.91%
  [13/84] Hidden=24 → RMSE: 4.88%
  [14/84] Hidden=28 → RMSE: 4.69%

Lag c

In [ ]:
print("\n" + "="*60)
print("STEP 7: Generating Table 1 (RMSE Summary)")
print("="*60)

table_data = {
    'RMSE on validation': ['at peaks', 'at non-peaks', 'overall'],
    'Standard': [
        f"{best_standard['Peak_RMSE_Relative']:.2f}%",
        f"{best_standard['NonPeak_RMSE_Relative']:.2f}%",
        f"{best_standard['Relative_RMSE']:.2f}%"
    ],
    'LSTM': [
        f"{best_lstm['Peak_RMSE_Relative']:.2f}%",
        f"{best_lstm['NonPeak_RMSE_Relative']:.2f}%",
        f"{best_lstm['Relative_RMSE']:.2f}%"
    ]
}

table_df = pd.DataFrame(table_data)
print("\n" + table_df.to_string(index=False))

table_df.to_csv('table1_rmse_summary.csv', index=False)
print("\n✓ Saved: table1_rmse_summary.csv")




STEP 7: Generating Table 1 (RMSE Summary)

RMSE on validation Standard  LSTM
          at peaks   10.26% 6.88%
      at non-peaks    1.78% 1.10%
           overall    4.21% 2.79%

✓ Saved: table1_rmse_summary.csv


In [ ]:
print("\n" + "="*60)
print("STEP 8: Generating Figure 3 - Best Standard NN")
print("="*60)

fig, ax = plt.subplots(figsize=(14, 6))
dates = best_standard['dates']
actual = best_standard['y_test']
pred = best_standard['y_pred_test']

ax.plot(dates, actual, 'b-', label='Actual', linewidth=2, alpha=0.7)
ax.plot(dates, pred, 'r--', label='Predicted', linewidth=2, alpha=0.8)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Discharge (ft³/s)', fontsize=12)
ax.set_title(f'Figure 3. Best model of Standard feed-forward\n(m={best_standard["Lag_Precip"]}, n={best_standard["Lag_Discharge"]}, hidden={best_standard["Hidden_Size"]}, RMSE={best_standard["Relative_RMSE"]:.2f}%)', fontsize=13)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('figure3_best_standard_nn.png', dpi=300, bbox_inches='tight')
print("✓ Saved: figure3_best_standard_nn.png")
plt.close()



STEP 8: Generating Figure 3 - Best Standard NN
✓ Saved: figure3_best_standard_nn.png


In [ ]:
print("\n" + "="*60)
print("STEP 9: Generating Figure 4 - Best LSTM")
print("="*60)

fig, ax = plt.subplots(figsize=(14, 6))
dates = best_lstm['dates']
actual = best_lstm['y_test']
pred = best_lstm['y_pred_test']

ax.plot(dates, actual, 'b-', label='Actual', linewidth=2, alpha=0.7)
ax.plot(dates, pred, 'r--', label='Predicted', linewidth=2, alpha=0.8)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Discharge (ft³/s)', fontsize=12)
ax.set_title(f'Figure 4. Best model of LSTM\n(m=n={best_lstm["Lag_Precip"]}, hidden={best_lstm["Hidden_Size"]}, RMSE={best_lstm["Relative_RMSE"]:.2f}%)', fontsize=13)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('figure4_best_lstm.png', dpi=300, bbox_inches='tight')
print("✓ Saved: figure4_best_lstm.png")
plt.close()


STEP 9: Generating Figure 4 - Best LSTM
✓ Saved: figure4_best_lstm.png


In [ ]:
print("\n" + "="*60)
print("STEP 10: Generating Figures 5-7 - RMSE Variation (Standard NN)")
print("="*60)

# Group by n-m difference
diff_groups = {1: [], 2: [], 3: []}
for _, row in results_standard_df.iterrows():
    diff = row['Lag_Discharge'] - row['Lag_Precip']
    if diff in diff_groups:
        diff_groups[diff].append(row.to_dict())

fig_count = 5
for diff, group_list in sorted(diff_groups.items()):
    if not group_list:
        continue

    group_df = pd.DataFrame(group_list)
    unique_m = sorted(group_df['Lag_Precip'].unique())

    fig, axes = plt.subplots(3, 1, figsize=(10, 12))

    metrics = [
        ('Relative_RMSE', axes[0], f'Overall RMSE (n-m={diff})'),
        ('Peak_RMSE_Relative', axes[1], f'RMSE at peaks (n-m={diff})'),
        ('NonPeak_RMSE_Relative', axes[2], f'RMSE at non-peaks (n-m={diff})')
    ]

    colors = ['blue', 'green', 'red', 'orange', 'purple', 'brown']

    for col, ax, title in metrics:
        for i, m in enumerate(unique_m):
            subset = group_df[group_df['Lag_Precip'] == m].sort_values('Hidden_Size')
            if len(subset) > 0:
                ax.plot(subset['Hidden_Size'], subset[col],
                       marker='o', label=f'm={m}',
                       color=colors[i % len(colors)], linewidth=2)

        ax.set_xlabel('Hidden size', fontsize=11)
        ax.set_ylabel('RMSE relative to max flow (%)', fontsize=11)
        ax.set_title(title, fontsize=12)
        ax.legend(fontsize=10)
        ax.grid(True, alpha=0.3)

    plt.suptitle(f'Figure {fig_count}. RMSE variation for the standard NN with n-m={diff}',
                 fontsize=14, y=0.995)
    plt.tight_layout()
    plt.savefig(f'figure{fig_count}_rmse_standard_diff{diff}.png', dpi=300, bbox_inches='tight')
    print(f"✓ Saved: figure{fig_count}_rmse_standard_diff{diff}.png")
    plt.close()
    fig_count += 1



STEP 10: Generating Figures 5-7 - RMSE Variation (Standard NN)
✓ Saved: figure5_rmse_standard_diff1.png
✓ Saved: figure6_rmse_standard_diff2.png
✓ Saved: figure7_rmse_standard_diff3.png


In [ ]:
print("\n" + "="*60)
print("STEP 11: Generating Figure 8 - RMSE Variation (LSTM)")
print("="*60)

unique_lags = sorted(results_lstm_df['Lag_Precip'].unique())

fig, axes = plt.subplots(3, 1, figsize=(10, 12))

metrics = [
    ('Relative_RMSE', axes[0], 'Overall RMSE'),
    ('Peak_RMSE_Relative', axes[1], 'RMSE at peaks'),
    ('NonPeak_RMSE_Relative', axes[2], 'RMSE at non-peaks')
]

colors = ['blue', 'green', 'red', 'orange', 'purple', 'brown']

for col, ax, title in metrics:
    for i, lag in enumerate(unique_lags):
        subset = results_lstm_df[results_lstm_df['Lag_Precip'] == lag].sort_values('Hidden_Size')
        if len(subset) > 0:
            ax.plot(subset['Hidden_Size'], subset[col],
                   marker='o', label=f'm=n={lag}',
                   color=colors[i % len(colors)], linewidth=2)

    ax.set_xlabel('Hidden size', fontsize=11)
    ax.set_ylabel('RMSE relative to max flow (%)', fontsize=11)
    ax.set_title(title, fontsize=12)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

plt.suptitle('Figure 8. RMSE variation for the LSTM network', fontsize=14, y=0.995)
plt.tight_layout()
plt.savefig('figure8_rmse_lstm.png', dpi=300, bbox_inches='tight')
print("✓ Saved: figure8_rmse_lstm.png")
plt.close()


STEP 11: Generating Figure 8 - RMSE Variation (LSTM)
✓ Saved: figure8_rmse_lstm.png


In [ ]:
print("\n" + "="*60)
print("STEP 12: Saving Detailed Results")
print("="*60)

# Save all results to CSV for further analysis
results_standard_summary = results_standard_df[[
    'Lag_Precip', 'Lag_Discharge', 'Hidden_Size',
    'Relative_RMSE', 'Peak_RMSE_Relative', 'NonPeak_RMSE_Relative',
    'Test_R2', 'best_epoch'
]].copy()

results_lstm_summary = results_lstm_df[[
    'Lag_Precip', 'Lag_Discharge', 'Hidden_Size',
    'Relative_RMSE', 'Peak_RMSE_Relative', 'NonPeak_RMSE_Relative',
    'Test_R2', 'best_epoch'
]].copy()

results_standard_summary.to_csv('detailed_results_standard_nn.csv', index=False)
results_lstm_summary.to_csv('detailed_results_lstm.csv', index=False)

print("✓ Saved: detailed_results_standard_nn.csv")
print("✓ Saved: detailed_results_lstm.csv")


STEP 12: Saving Detailed Results
✓ Saved: detailed_results_standard_nn.csv
✓ Saved: detailed_results_lstm.csv


In [ ]:
print("\n" + "="*80)
print("IMPLEMENTATION COMPLETE! ✓")
print("="*80)

print(f"\n📁 Generated Outputs:")
print(f"   1. table1_rmse_summary.csv - Table 1: RMSE comparison")
print(f"   2. figure3_best_standard_nn.png - Best Standard NN predictions")
print(f"   3. figure4_best_lstm.png - Best LSTM predictions")
print(f"   4. figure5_rmse_standard_diff1.png - RMSE variation (n-m=1)")
print(f"   5. figure6_rmse_standard_diff2.png - RMSE variation (n-m=2)")
print(f"   6. figure7_rmse_standard_diff3.png - RMSE variation (n-m=3)")
print(f"   7. figure8_rmse_lstm.png - RMSE variation for LSTM")
print(f"   8. detailed_results_standard_nn.csv - All Standard NN results")
print(f"   9. detailed_results_lstm.csv - All LSTM results")

print(f"\n🔍 Diagnostic Information:")
print(f"   Total Standard NN models trained: {len(results_standard_df)}")
print(f"   Total LSTM models trained: {len(results_lstm_df)}")
print(f"   Average epochs to convergence (Standard NN): {results_standard_df['best_epoch'].mean():.1f}")
print(f"   Average epochs to convergence (LSTM): {results_lstm_df['best_epoch'].mean():.1f}")

print(f"\n📊 Performance Comparison with Paper:")
print(f"\n   Standard NN:")
print(f"      Your results  → Overall: {best_standard['Relative_RMSE']:.2f}%, Peak: {best_standard['Peak_RMSE_Relative']:.2f}%, Non-Peak: {best_standard['NonPeak_RMSE_Relative']:.2f}%")
print(f"      Paper results → Overall: 0.23%, Peak: 0.63%, Non-Peak: 0.07%")
print(f"      Ratio         → Overall: {best_standard['Relative_RMSE']/0.23:.1f}x, Peak: {best_standard['Peak_RMSE_Relative']/0.63:.1f}x, Non-Peak: {best_standard['NonPeak_RMSE_Relative']/0.07:.1f}x")

print(f"\n   LSTM:")
print(f"      Your results  → Overall: {best_lstm['Relative_RMSE']:.2f}%, Peak: {best_lstm['Peak_RMSE_Relative']:.2f}%, Non-Peak: {best_lstm['NonPeak_RMSE_Relative']:.2f}%")
print(f"      Paper results → Overall: 2.43%, Peak: 6.78%, Non-Peak: 0.54%")
print(f"      Ratio         → Overall: {best_lstm['Relative_RMSE']/2.43:.2f}x, Peak: {best_lstm['Peak_RMSE_Relative']/6.78:.2f}x, Non-Peak: {best_lstm['NonPeak_RMSE_Relative']/0.54:.2f}x")

print(f"\n💡 Key Improvements Applied:")
print(f"   ✓ Removed early stopping - all models train for full 200 epochs")
print(f"   ✓ Using SGD optimizer for Standard NN (lr=0.009)")
print(f"   ✓ Using Adam optimizer for LSTM (lr=0.001)")
print(f"   ✓ Proper best model selection via ModelCheckpoint")
print(f"   ✓ Correct activation functions (sigmoid hidden, linear output)")
print(f"   ✓ Peak threshold applied to denormalized values (1500 ft³/s)")

print(f"\n⚠️ If results still differ significantly from paper:")
print(f"   1. Check if your discharge_max matches paper's 29,700 ft³/s")
print(f"   2. Check if your precipitation_max matches paper's 6.8 inches")
print(f"   3. Verify data comes from Leaf River Basin near Collins, LA")
print(f"   4. Ensure time period is similar (paper: 2003-2012)")
print(f"   5. Consider that paper may have used different preprocessing")
print(f"   6. Random initialization may cause slight variations")

print(f"\n🎯 Next Steps:")
print(f"   • Review the generated figures to visually assess predictions")
print(f"   • Compare RMSE patterns with paper's Figures 5-8")
print(f"   • If Standard NN still underperforms, try:")
print(f"     - Different batch sizes (try 16 or 64)")
print(f"     - Different random seeds")
print(f"     - Verify normalization is correct")
print(f"     - Check for data quality issues")

print("\n" + "="*80)
print("END OF IMPLEMENTATION")
print("="*80)


IMPLEMENTATION COMPLETE! ✓

📁 Generated Outputs:
   1. table1_rmse_summary.csv - Table 1: RMSE comparison
   2. figure3_best_standard_nn.png - Best Standard NN predictions
   3. figure4_best_lstm.png - Best LSTM predictions
   4. figure5_rmse_standard_diff1.png - RMSE variation (n-m=1)
   5. figure6_rmse_standard_diff2.png - RMSE variation (n-m=2)
   6. figure7_rmse_standard_diff3.png - RMSE variation (n-m=3)
   7. figure8_rmse_lstm.png - RMSE variation for LSTM
   8. detailed_results_standard_nn.csv - All Standard NN results
   9. detailed_results_lstm.csv - All LSTM results

🔍 Diagnostic Information:
   Total Standard NN models trained: 84
   Total LSTM models trained: 42
   Average epochs to convergence (Standard NN): 200.0
   Average epochs to convergence (LSTM): 78.5

📊 Performance Comparison with Paper:

   Standard NN:
      Your results  → Overall: 4.21%, Peak: 10.26%, Non-Peak: 1.78%
      Paper results → Overall: 0.23%, Peak: 0.63%, Non-Peak: 0.07%
      Ratio         → Over